# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import hvplot
import pandas as pd
import requests
import matplotlib

# Import API key
from api_keys import geoapify_key

/Users/samwallach/Documents/BootCamp/Module_3_Python/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ierapetra,35.0100,25.7353,19.52,53,76,4.92,GR,1701875576
1,1,sardinata,8.0829,-72.8007,31.82,46,60,1.55,CO,1701875577
2,2,arraial do cabo,-22.9661,-42.0278,29.92,62,0,8.75,BR,1701875577
3,3,ushuaia,-54.8000,-68.3000,10.79,73,74,5.14,AR,1701875577
4,4,afaahiti,-17.7500,-149.2833,22.93,86,100,7.14,PF,1701875578


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    hover_cols = ["Country", "Max Temp"]
)
# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = pd.DataFrame(city_data_df.loc[((city_data_df["Max Temp"]>=20) &
                                                  (city_data_df["Max Temp"]<=30) &
                                                  (city_data_df["Wind Speed"]<=5) &
                                                  (city_data_df["Cloudiness"]<=3)),:])
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,al hufuf,25.3646,49.5653,21.19,23,0,1.54,SA,1701875606
103,103,bubaque,11.2833,-15.8333,28.50,39,2,2.44,GW,1701875626
150,150,salalah,17.0151,54.0924,27.05,57,0,1.54,OM,1701875639
158,158,alice springs,-23.7000,133.8833,23.75,46,1,2.57,AU,1701875642
181,181,al kharijah,25.4514,30.5464,20.87,32,0,3.42,EG,1701875651
203,203,ha'il,27.5219,41.6907,23.01,25,0,2.06,SA,1701875659
206,206,long beach,33.7670,-118.1892,22.06,69,0,0.00,US,1701875660
255,255,pisco,-13.7000,-76.2167,23.03,73,0,4.12,PE,1701875678
277,277,recani,31.2846,-4.2688,21.12,27,3,0.94,MA,1701875685
286,286,vallenar,-28.5708,-70.7581,26.89,26,0,3.01,CL,1701875687


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
del hotel_df["City_ID"]
del hotel_df["Max Temp"]
del hotel_df["Cloudiness"]
del hotel_df["Wind Speed"]
del hotel_df["Date"]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
79,al hufuf,25.3646,49.5653,23,SA,
103,bubaque,11.2833,-15.8333,39,GW,
150,salalah,17.0151,54.0924,57,OM,
158,alice springs,-23.7000,133.8833,46,AU,
181,al kharijah,25.4514,30.5464,32,EG,
203,ha'il,27.5219,41.6907,25,SA,
206,long beach,33.7670,-118.1892,69,US,
255,pisco,-13.7000,-76.2167,73,PE,
277,recani,31.2846,-4.2688,27,MA,
286,vallenar,-28.5708,-70.7581,26,CL,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
bubaque - nearest hotel: Chez Julio
salalah - nearest hotel: Muscat International Hotel
alice springs - nearest hotel: Aurora Alice Springs
al kharijah - nearest hotel: Kharga Hotel
ha'il - nearest hotel: فندق كلاسيك
long beach - nearest hotel: Westin
pisco - nearest hotel: La Portada
recani - nearest hotel: Hotel Merzouga
vallenar - nearest hotel: Hotel del Marqués
kailua-kona - nearest hotel: Kona Seaside Hotel
freetown - nearest hotel: Formal Leona hotel
diego de almagro - nearest hotel: Residencial Hambyll
sarmiento - nearest hotel: Ismar


,City,Lat,Lng,Humidity,Country,Hotel Name
79,al hufuf,25.3646,49.5653,23,SA,Al Muhaidab Residence Al Ahsa
103,bubaque,11.2833,-15.8333,39,GW,Chez Julio
150,salalah,17.0151,54.0924,57,OM,Muscat International Hotel
158,alice springs,-23.7000,133.8833,46,AU,Aurora Alice Springs
181,al kharijah,25.4514,30.5464,32,EG,Kharga Hotel
203,ha'il,27.5219,41.6907,25,SA,فندق كلاسيك
206,long beach,33.7670,-118.1892,69,US,Westin
255,pisco,-13.7000,-76.2167,73,PE,La Portada
277,recani,31.2846,-4.2688,27,MA,Hotel Merzouga
286,vallenar,-28.5708,-70.7581,26,CL,Hotel del Marqués


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot

hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)